In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load the data
df = pd.read_csv(
    "/Users/macintosh/Desktop/Dissertation/Smart-Inventory-Replenishment-System-for-E-Commerce/EDA/Model_data"
)

In [7]:
df.head(5)

,Order ID,Order Item ID,Order YearMonth,Order Day,Order Time,Order Quantity,Product Department,Product Category,Product Name,Customer ID,...,Daily Demand Std Dev,Safety Stock,Season,Season Encoded,Customer Market Encoded,Customer Region Encoded,Shipment Mode Encoded,Warehouse Country Encoded,day_of_week,month
0,1,4381,202101,1970-01-01 00:00:00.000000001,2:48,1,Fan Shop,Water Sports,Pelican Sunstream 100 Kayak,7399,...,1.510322,4.341208,Winter,0,2,3,0,0,4,1
1,2,29,202101,1970-01-01 00:00:00.000000001,3:30,1,Apparel,Cleats,Perfect Fitness Perfect Rip Deck,918,...,1.510322,4.361511,Winter,0,2,12,3,1,4,1
2,3,32,202101,1970-01-01 00:00:00.000000001,3:30,4,Footwear,Cardio Equipment,Nike Men's Free 5.0+ Running Shoe,918,...,1.510322,4.367121,Winter,0,2,12,3,1,4,1
3,4,33,202101,1970-01-01 00:00:00.000000001,3:30,5,Fan Shop,Indoor/Outdoor Games,O'Brien Men's Neoprene Life Vest,918,...,1.510322,4.386721,Winter,0,2,12,3,1,4,1
4,5,109801,202101,1970-01-01 00:00:00.000000001,4:12,5,Footwear,Electronics,Under Armour Women's Ignite Slide,6124,...,1.510322,4.264630,Winter,0,1,8,3,0,4,1


In [ ]:
# aggregate order quantity by product to get daily demand
daily_demand = df.groupby(['date', 'product_id'])['order_quantity'].sum().reset_index()

In [3]:
# drop missing values
df.dropna(
    subset=["Daily Demand", "Warehouse Inventory", "Lead Time", "Safety Stock"],
    inplace=True,
)

In [4]:
# Convert dates to datetime
df["date"] = pd.to_datetime(df["date"])
df["Order Day"] = pd.to_datetime(df["Order Day"])
df["Shipment Day"] = pd.to_datetime(df["Shipment Day"])
df["day_of_week"] = df["date"].dt.dayofweek
df["month"] = df["date"].dt.month

In [6]:
# Forecast daily demand
features_demand = [
    col
    for col in df.columns
    if col
    not in [
        "Daily Demand",
        "Stockout",
        "Restock Now",
        "Order Time",
        "Order Day",
        "Shipment Day",
        "date",
        "Order Quantity",
        "Gross Sales",
        "Profit",
        "Profit Margin",
        "Inventory Cost",
        "Inventory Efficiency",
        "Estimated Ordering Cost",
        "EOQ",
        "Reorder Point",
        "Inventory Value",
        "Ordering Cost",
        "Holding Cost",
        "Holding Rate",
        "days of supply",
    ]
]
X_demand = df[features_demand]
y_demand = df["Daily Demand"]

X_train, X_test, y_train, y_test = train_test_split(
    X_demand, y_demand, test_size=0.2, random_state=42
)
model_demand = RandomForestRegressor(n_estimators=100, random_state=42)
model_demand.fit(X_train, y_train)
df["Predicted_Daily_Demand"] = model_demand.predict(X_demand)

ValueError: could not convert string to float: 'Fan Shop'